<a href="https://colab.research.google.com/github/Austfi/SNOWPACKforPatrollers/blob/dev/Snowprofile_Colab_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Snowprofile Tutorial: Read, Plot, and Export CAAML Files

This tutorial teaches you how to use the `snowprofile` Python package with CAAML v6 files. You'll learn to read snow profile data, explore the data structure, create plots, and export back to CAAML format.

**What you'll do:**
1. Install the package
2. Load a CAAML file
3. Explore the data
4. Create plots
5. Export the results

**Example file**: This tutorial uses `snowprofiles/example_profile.caaml`. When running in Colab, it will automatically download from the repository.



In [ ]:
# Step 1: Setup environment
# This cell checks if you're running in Google Colab and sets up plotting

import sys

# Check if running in Colab
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    print("✓ Running in Google Colab")
else:
    print("⚠ Not running in Colab - some features may not work")

# Silence warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Enable inline plotting (shows plots in notebook)
%matplotlib inline
print("✓ Setup complete")




⚠ Not running in Colab - some features may not work
✓ Matplotlib backend: Agg


In [ ]:
# Step 2: Install the snowprofile package
# Run this cell to install the package and dependencies

%pip install -q snowprofile pandas numpy matplotlib

# Verify installation
import snowprofile

# Check version
try:
    version = snowprofile.__version__
    print(f"✓ snowprofile version: {version}")
except AttributeError:
    try:
        import importlib.metadata
        version = importlib.metadata.version('snowprofile')
        print(f"✓ snowprofile version: {version}")
    except Exception:
        print("✓ snowprofile installed and ready to use")





Note: you may need to restart the kernel to use updated packages.


AttributeError: module 'snowprofile' has no attribute '__version__'

In [ ]:
# Step 3: Import required libraries
# This loads the tools we need for working with snow profiles

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import snowprofile functions
import snowprofile
from snowprofile import io as spio  # For reading/writing CAAML files
from snowprofile import plot as spplot  # For plotting

print("✓ All libraries imported successfully")




In [ ]:
# Step 4: Load your CAAML file
# This cell loads the example file. To use your own file, change CAAML_PATH below.

# Try to use the example file from the repository
EXAMPLE_FILE = Path("snowprofiles/example_profile.caaml")

if EXAMPLE_FILE.exists():
    CAAML_PATH = EXAMPLE_FILE
    print(f"✓ Using example file: {CAAML_PATH}")
elif IN_COLAB:
    # Download example file if running in Colab
    print("Downloading example CAAML file...")
    import urllib.request
    
    # Try dev branch first (where the file is), then main as fallback
    branches = ["dev", "main"]
    downloaded = False
    
    for branch in branches:
        try:
            example_url = f"https://raw.githubusercontent.com/Austfi/SNOWPACKforPatrollers/{branch}/snowprofiles/example_profile.caaml"
            CAAML_PATH = Path("example_profile.caaml")
            urllib.request.urlretrieve(example_url, str(CAAML_PATH))
            print(f"✓ Downloaded from {branch} branch: {CAAML_PATH}")
            downloaded = True
            break
        except Exception as e:
            continue
    
    if not downloaded:
        # Fallback: file upload
        print("⚠ Could not download example file. Please upload your own CAAML file:")
        from google.colab import files
        uploaded = files.upload()
        CAAML_PATH = Path(next(iter(uploaded)))
        print(f"✓ Uploaded: {CAAML_PATH}")
else:
    # For your own file, change this path:
    CAAML_PATH = Path("your_file.caaml")  # ← Change this to your file path
    print(f"Using file: {CAAML_PATH}")

# Verify file exists
assert CAAML_PATH.exists(), f"File not found: {CAAML_PATH}"
print(f"✓ File ready: {CAAML_PATH}")







In [ ]:
## Step 5: Read the CAAML File

This reads your CAAML file and creates a SnowProfile object that contains all the data.



In [ ]:
# Read the CAAML file
# This creates a SnowProfile object called 'sp' that contains all your data

sp = spio.read_caaml6_xml(str(CAAML_PATH))
print(f"✓ Successfully loaded: {CAAML_PATH.name}")
print(f"✓ SnowProfile object created")

# The variable 'sp' now contains all your snow profile data
# You can use 'sp' in the cells below to explore and plot the data



In [ ]:
## Step 6: Explore Your Snow Profile Data

The SnowProfile object (`sp`) contains information about:
- **Time**: When the profile was recorded
- **Location**: Where it was taken (name, elevation, coordinates)
- **Profile depth**: Total snow depth in meters
- **Stratigraphy**: Information about each snow layer
- **Profiles**: Temperature, density, hardness measurements at different depths



# View basic information about your snow profile

print("=" * 60)
print("SNOW PROFILE INFORMATION")
print("=" * 60)

# Time
if hasattr(sp, 'record_time') and sp.record_time:
    print(f"\nRecord time: {sp.record_time}")

# Location
if hasattr(sp, 'name') and sp.name:
    print(f"Location: {sp.name}")
if hasattr(sp, 'elevation') and sp.elevation is not None:
    print(f"Elevation: {sp.elevation} m")
if hasattr(sp, 'latitude') and sp.latitude is not None:
    print(f"Coordinates: {sp.latitude}°N, {sp.longitude}°W")

# Profile depth
if hasattr(sp, 'profile_depth') and sp.profile_depth is not None:
    print(f"\nProfile depth: {sp.profile_depth:.2f} m")
if hasattr(sp, 'profile_swe') and sp.profile_swe is not None:
    print(f"Snow Water Equivalent (SWE): {sp.profile_swe:.1f} mm")

# Available data types
print(f"\nAvailable data:")
print(f"  Temperature profiles: {len(sp.temperature_profiles) if sp.temperature_profiles else 0}")
print(f"  Density profiles: {len(sp.density_profiles) if sp.density_profiles else 0}")
print(f"  Stratigraphy layers: {len(sp.stratigraphy_profile.data) if sp.stratigraphy_profile and hasattr(sp.stratigraphy_profile, 'data') else 'not available'}")

print("=" * 60)



In [ ]:
# Look at the stratigraphy (snow layers)
# This shows each layer with its properties like thickness, grain type, and hardness

if sp.stratigraphy_profile is not None:
    # Get the data as a DataFrame (like a spreadsheet)
    df = sp.stratigraphy_profile.data
    print(f"Total layers: {len(df)}")
    print(f"\nFirst few layers:")
    print(df.head())
    
    # Show layer thicknesses
    if 'thickness' in df.columns:
        print(f"\nLayer thickness summary:")
        print(f"  Average: {df['thickness'].mean():.3f} m")
        print(f"  Thickest: {df['thickness'].max():.3f} m")
        print(f"  Thinnest: {df['thickness'].min():.3f} m")
else:
    print("No stratigraphy data available")




In [ ]:
## Step 7: Analyze Temperature Profile

Temperature gradients show how quickly temperature changes with depth. Strong gradients (≥ 10 °C/m) can indicate rapid temperature changes or potential instability.

**Formula**: The gradient is calculated as change in temperature divided by change in depth: dT/dz ≈ (T₂ - T₁) / (z₂ - z₁)

**Units**: °C/m (degrees Celsius per meter)



# Analyze temperature profile and calculate gradients
# Change temp_profiles[0] to temp_profiles[1] if you want to analyze a different temperature profile

if sp.temperature_profiles and len(sp.temperature_profiles) > 0:
    # Get the first temperature profile
    temp_profile = sp.temperature_profiles[0]  # ← Change index [0] to use a different profile
    
    # Get the data
    temp_data = temp_profile.data
    
    # Sort by height (depth)
    temp_data = temp_data.sort_values('height')
    heights = temp_data['height'].values
    temps = temp_data['temperature'].values
    
    # Calculate temperature gradient (how fast temperature changes with depth)
    dTdz = np.gradient(temps, heights)
    
    # Find strong gradients (≥ 10 °C/m)
    strong_gradients = np.abs(dTdz) >= 10
    
    print(f"Temperature profile analysis:")
    print(f"  Data points: {len(heights)}")
    print(f"  Depth range: {heights.min():.2f} - {heights.max():.2f} m")
    print(f"  Temperature range: {temps.min():.2f} - {temps.max():.2f} °C")
    print(f"  Strong gradients (≥10 °C/m): {np.sum(strong_gradients)} points")
    
    # Plot temperature and gradient
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
    
    # Temperature profile
    ax1.plot(temps, heights, 'o-', markersize=5)
    ax1.set_xlabel('Temperature (°C)')
    ax1.set_ylabel('Height (m)')
    ax1.set_title('Temperature Profile')
    ax1.grid(True, alpha=0.3)
    
    # Temperature gradient
    ax2.plot(dTdz, heights, 'o-', markersize=5, color='orange')
    ax2.axvline(x=0, color='k', linestyle='--', alpha=0.3)
    ax2.axvline(x=10, color='r', linestyle='--', alpha=0.5, label='±10 °C/m')
    ax2.axvline(x=-10, color='r', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Temperature Gradient (°C/m)')
    ax2.set_ylabel('Height (m)')
    ax2.set_title('Temperature Gradient')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No temperature profile available")



In [ ]:
# Temperature gradient analysis

if hasattr(sp, 'temperature_profiles') and sp.temperature_profiles:
    temp_profiles = sp.temperature_profiles
    if len(temp_profiles) > 0:
        # Use first temperature profile
        temp_profile = temp_profiles[0]
        
        # Get data
        if hasattr(temp_profile, 'data'):
            temp_data = temp_profile.data
        elif hasattr(temp_profile, 'data_dict'):
            temp_data = pd.DataFrame(temp_profile.data_dict)
        else:
            temp_data = None
        
        if temp_data is not None and not temp_data.empty:
            # Ensure we have height and temperature columns
            if 'height' in temp_data.columns and 'temperature' in temp_data.columns:
                # Sort by height
                temp_data = temp_data.sort_values('height')
                heights = temp_data['height'].values
                temps = temp_data['temperature'].values
                
                # Remove NaN values
                valid_mask = ~(np.isnan(heights) | np.isnan(temps))
                heights = heights[valid_mask]
                temps = temps[valid_mask]
                
                if len(heights) > 1:
                    # Compute gradient using numpy's gradient function
                    # edge_order=2 uses second-order accurate differences at boundaries
                    dTdz = np.gradient(temps, heights, edge_order=2)
                    
                    # Create strong gradient flag (≥ 10 °C/m threshold)
                    strong_gradient_flag = np.abs(dTdz) >= 10
                    
                    print(f"✓ Temperature profile analyzed")
                    print(f"  Data points: {len(heights)}")
                    print(f"  Height range: {heights.min():.2f} - {heights.max():.2f} m")
                    print(f"  Temperature range: {temps.min():.2f} - {temps.max():.2f} °C")
                    print(f"  Gradient range: {dTdz.min():.2f} - {dTdz.max():.2f} °C/m")
                    print(f"  Strong gradients (≥10 °C/m): {np.sum(strong_gradient_flag)} points")
                    
                    # Plot temperature vs height
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
                    
                    # Temperature profile
                    ax1.plot(temps, heights, 'o-', markersize=4)
                    ax1.set_xlabel('Temperature (°C)')
                    ax1.set_ylabel('Height (m)')
                    ax1.set_title('Temperature Profile')
                    ax1.grid(True, alpha=0.3)
                    ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)
                    
                    # Temperature gradient
                    ax2.plot(dTdz, heights, 'o-', markersize=4, color='orange')
                    ax2.axvline(x=0, color='k', linestyle='--', alpha=0.3)
                    ax2.axvline(x=10, color='r', linestyle='--', alpha=0.5, label='±10 °C/m threshold')
                    ax2.axvline(x=-10, color='r', linestyle='--', alpha=0.5)
                    ax2.set_xlabel('Temperature Gradient (°C/m)')
                    ax2.set_ylabel('Height (m)')
                    ax2.set_title('Temperature Gradient')
                    ax2.grid(True, alpha=0.3)
                    ax2.legend()
                    
                    plt.tight_layout()
                    plt.show()
                    
                    print("\nNote: Heights are in meters, zero at bottom. The library handles unit conversions automatically.")
                else:
                    print("⚠ Insufficient data points for gradient calculation")
            else:
                print(f"⚠ Missing required columns. Available: {list(temp_data.columns)}")
        else:
            print("⚠ Temperature data not available")
    else:
        print("⚠ No temperature profiles available")
else:
    print("⚠ Temperature profile data not available")



In [ ]:
# Create a simple plot of your snow profile
# This shows the stratigraphy and available profiles

spplot.plot_simple(sp)
plt.show()
print("✓ Plot created")




In [ ]:
# Export your SnowProfile to a CAAML file
# Change the filename and version as needed

# Create output directory
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Export to CAAML 6.0.5 (change version to "6.0.6" if needed)
output_file = output_dir / "profile_out.caaml"  # ← Change filename here if desired
spio.write_caaml6_xml(sp, str(output_file), version="6.0.5", indent=True)

print(f"✓ Exported to: {output_file}")

# Download file if running in Colab
if IN_COLAB:
    from google.colab import files
    files.download(str(output_file))
    print("✓ File downloaded")




In [ ]:
## Step 9: Export Back to CAAML

Save your SnowProfile object back to a CAAML file. You can change the output filename and version as needed.



In [ ]:
# Export your SnowProfile to a CAAML file
# Change the filename and version as needed

# Create output directory
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Export to CAAML 6.0.5 (change version to "6.0.6" if needed)
output_file = output_dir / "profile_out.caaml"  # ← Change filename here if desired
spio.write_caaml6_xml(sp, str(output_file), version="6.0.5", indent=True)

print(f"✓ Exported to: {output_file}")

# Download file if running in Colab
if IN_COLAB:
    from google.colab import files
    files.download(str(output_file))
    print("✓ File downloaded")




## Tutorial Complete!

You've learned how to:
- Load a CAAML file into a SnowProfile object
- Explore the data (time, location, layers, profiles)
- Analyze temperature gradients
- Create visualizations
- Export back to CAAML format

**To use your own file**: Change `CAAML_PATH` in Step 4 to point to your CAAML file.

**To modify the analysis**: Edit the code cells above. Each cell has comments explaining what it does and where you can make changes.
